In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [33]:
# CUDA, 실행 device setting
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [34]:
EPOCHS = 30
BATCH_SIZE = 64

In [35]:
# 이미지를 tensor로 바꿔주는 코드
transform = transforms.Compose([
    transforms.ToTensor()
])

In [36]:
# Fashion MNIST dataset download
trainset = datasets.FashionMNIST(
    root = './.data/',
    train = True,
    download = True,
    transform = transform
)
testset = datasets.FashionMNIST(
    root = './.data/',
    train = False,
    download = True,
    transform = transform
)

In [37]:
train_loader = torch.utils.data.DataLoader(
    dataset = trainset,
    batch_size = BATCH_SIZE,
    shuffle = True
)
test_loader = torch.utils.data.DataLoader(
    dataset = testset,
    batch_size = BATCH_SIZE,
    shuffle = True
)

In [38]:
# ANN settings
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        
    def forward(self, x):
        # 1차원 행렬로 변환
        x = x.view(-1, 784)
        # 각 레이어를 거쳐 활성화 함수 실행, 다음 뉴런으로 진행
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [39]:
# 설정한 실행 환경에 모델 선언
model = Net().to(DEVICE)

# 최적화 알고리즘 설정
optimizer = optim.SGD(model.parameters(), lr = 0.01)

In [1]:
# 학습 함수
def train(model, train_loader, optimizer):
    model.train()
    
    for batch_dix, (data, target) in enumerate(train_loader):
        # 학습 데이터를 DEVICE의 메모리로 보냄
        data, target = data.to(DEVICE), target.to(DEVICE)
        
        optimizer.zero_grad()
        output = model(data)
        # loss function 설정, loss 값을 저장하지 않을 때는 바로 functional에서 호출해서 사용 가능
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [41]:
# 평가 함수 
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0 # 예측을 맞춘 수를 저장하기 위한 변수
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            # 모든 오차 더하기
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            # 가장 큰 값을 가진 클래스가 모델의 예측
            # 예측과 정답을 비교하여 일치할 경우 correct에 1을 더한다.
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
        # 정확도 측정
        test_loss /= len(test_loader.dataset)
        test_accuracy = 100. * correct / len(test_loader.dataset)
        return test_loss, test_accuracy

In [42]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'
          .format(epoch, test_loss, test_accuracy))

[1] Test Loss: 0.8370, Accuracy: 69.18%
[2] Test Loss: 0.6653, Accuracy: 77.01%
[3] Test Loss: 0.5853, Accuracy: 79.39%
[4] Test Loss: 0.5385, Accuracy: 80.99%
[5] Test Loss: 0.5120, Accuracy: 81.86%
[6] Test Loss: 0.5016, Accuracy: 82.51%
[7] Test Loss: 0.4926, Accuracy: 82.65%
[8] Test Loss: 0.5083, Accuracy: 82.10%
[9] Test Loss: 0.4723, Accuracy: 83.56%
[10] Test Loss: 0.4740, Accuracy: 82.80%
[11] Test Loss: 0.5268, Accuracy: 81.75%
[12] Test Loss: 0.4592, Accuracy: 83.80%
[13] Test Loss: 0.4349, Accuracy: 84.67%
[14] Test Loss: 0.4388, Accuracy: 84.55%
[15] Test Loss: 0.4522, Accuracy: 84.18%
[16] Test Loss: 0.4230, Accuracy: 85.06%
[17] Test Loss: 0.4179, Accuracy: 85.13%
[18] Test Loss: 0.4172, Accuracy: 85.44%
[19] Test Loss: 0.3992, Accuracy: 85.94%
[20] Test Loss: 0.4119, Accuracy: 85.47%
[21] Test Loss: 0.4089, Accuracy: 85.67%
[22] Test Loss: 0.4369, Accuracy: 84.23%
[23] Test Loss: 0.3853, Accuracy: 86.49%
[24] Test Loss: 0.4095, Accuracy: 85.49%
[25] Test Loss: 0.3847, A